<a href="https://colab.research.google.com/github/franciscogonzalez-gal/text_image_recognition/blob/main/Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ============================================
# Laboratorio #4 — Text Mining & Image Recognition
# Autor: Francisco Gonzalez
# Carnet: 24002914
# ============================================

import re
import os
import glob

In [10]:
# ============================================
# Montar Google Drive (ejecuta esta celda en Colab)
# ============================================
from google.colab import drive
drive.mount('/content/drive')

# Ajusta esta ruta si tu carpeta está en otro lugar:
DATA_DIR = '/content/drive/MyDrive/datos/lab4'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# ============================================
# PROBLEMA 1: Expresiones regulares
# (Se imprimen como cadenas listas para entregar)
# ============================================

# 1) Email (general)
regex_email = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'

# 2) URL con protocolo http/https, "www." y TLD restringido a .com | .org | .edu
#    Dominio: inicia con letra, luego letras/dígitos/guiones.
regex_url = r'^(?:https?://)www\.[A-Za-z][A-Za-z0-9-]*(?:\.[A-Za-z0-9-]+)*\.(?:com|org|edu)$'

# 3) MAC Address (6 bloques hexadecimales en MAYÚSCULAS o minúsculas separados por espacio)
regex_mac = r'^(?:[0-9A-Fa-f]{2}\s){5}[0-9A-Fa-f]{2}$'

# 4) IPv4 (0–255 por octeto)
regex_ipv4 = (
    r'^(?:25[0-5]|2[0-4]\d|1?\d?\d)'
    r'(?:\.(?:25[0-5]|2[0-4]\d|1?\d?\d)){3}$'
)

# 5) Fecha d-m-a con separador / . o -, día 1–31, mes 1–12,
#    año 2000–2019 o bien 00–19 (dos dígitos)
regex_fecha_dmy = (
    r'^(?:[1-9]|[12]\d|3[01])'              # día
    r'[\/\-.]'                               # separador
    r'(?:[1-9]|1[0-2])'                      # mes
    r'[\/\-.]'                               # separador
    r'(?:(?:200\d|201\d)|(?:0\d|1\d))$'      # año: 2000–2019 o 00–19
)

# Imprimir (para entregar las "strings" de las regex)
print("Regex email:", regex_email)
print("Regex URL  :", regex_url)
print("Regex MAC  :", regex_mac)
print("Regex IPv4 :", regex_ipv4)
print("Regex fecha:", regex_fecha_dmy)

Regex email: ^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$
Regex URL  : ^(?:https?://)www\.[A-Za-z][A-Za-z0-9-]*(?:\.[A-Za-z0-9-]+)*\.(?:com|org|edu)$
Regex MAC  : ^(?:[0-9A-Fa-f]{2}\s){5}[0-9A-Fa-f]{2}$
Regex IPv4 : ^(?:25[0-5]|2[0-4]\d|1?\d?\d)(?:\.(?:25[0-5]|2[0-4]\d|1?\d?\d)){3}$
Regex fecha: ^(?:[1-9]|[12]\d|3[01])[\/\-.](?:[1-9]|1[0-2])[\/\-.](?:(?:200\d|201\d)|(?:0\d|1\d))$


In [12]:
#Pequeña validación rápida con los ejemplos del enunciado
ejemplos_email = [
    "Guate.360-porelmundo@miguate.com",
    "Miercoles3@hotmail.com",
    "Progra3.galileo@galileo.edu",
]
print("\nValidación rápida de emails:")
for s in ejemplos_email:
    print(s, "->", bool(re.fullmatch(regex_email, s)))

ejemplos_url = [
    "https://www.guate360-porelmundo.com",  # válido (.com)
    "http://www.a2.net",                    # inválido por TLD (.net no permitido)
    "https://www.galileo.edu",              # válido (.edu)
    "http://www.8.org",                     # inválido: dominio inicia con dígito
]
print("\nValidación rápida de URLs:")
for s in ejemplos_url:
    print(s, "->", bool(re.fullmatch(regex_url, s)))

ejemplos_mac = [
    "5A 6F AF 8C 9B 1D",
    "6D 6C 4D 3A EB 3F",
    "3A 7C FA C8 6D 4J",  # inválido (J)
]
print("\nValidación rápida de MACs:")
for s in ejemplos_mac:
    print(s, "->", bool(re.fullmatch(regex_mac, s)))

ejemplos_ipv4 = [
    "192.16.8.1",
    "234.56.78.90",
    "1.2.3.4",
    "192.168.45.345",  # inválido (>255)
]
print("\nValidación rápida de IPv4:")
for s in ejemplos_ipv4:
    print(s, "->", bool(re.fullmatch(regex_ipv4, s)))

ejemplos_fecha = [
    "20/1/2019",
    "12.03.2005",  # 2005 no está en 2000–2019 -> debería dar False
    "31-11-08",    # año 08 => 2008 (aceptado por la regex), mes 11, día 31 (no valida por calendario real, pero el enunciado pide validar rangos generales)
    "1-1-2012",
    "12-12-22",    # 22 (>19) → False
]
print("\nValidación rápida de fechas d-m-a (regex):")
for s in ejemplos_fecha:
    print(s, "->", bool(re.fullmatch(regex_fecha_dmy, s)))


Validación rápida de emails:
Guate.360-porelmundo@miguate.com -> True
Miercoles3@hotmail.com -> True
Progra3.galileo@galileo.edu -> True

Validación rápida de URLs:
https://www.guate360-porelmundo.com -> True
http://www.a2.net -> False
https://www.galileo.edu -> True
http://www.8.org -> False

Validación rápida de MACs:
5A 6F AF 8C 9B 1D -> True
6D 6C 4D 3A EB 3F -> True
3A 7C FA C8 6D 4J -> False

Validación rápida de IPv4:
192.16.8.1 -> True
234.56.78.90 -> True
1.2.3.4 -> True
192.168.45.345 -> False

Validación rápida de fechas d-m-a (regex):
20/1/2019 -> True
12.03.2005 -> False
31-11-08 -> True
1-1-2012 -> True
12-12-22 -> False


In [13]:
# ============================================
# PROBLEMA 2 (ajustado a archivos D1..D21)
# - Lee D*.txt en /MyDrive/datos/lab4
# - Extrae fechas dd/mm/aa(aa)
# - Calcula promedios de día, mes y año
# ============================================


# Mapeo de meses en inglés a número (abreviado o completo)
MONTHS_EN = {
    'jan': 1, 'january': 1,
    'feb': 2, 'february': 2,
    'mar': 3, 'march': 3,
    'apr': 4, 'april': 4,
    'may': 5,
    'jun': 6, 'june': 6,
    'jul': 7, 'july': 7,
    'aug': 8, 'august': 8,
    'sep': 9, 'sept': 9, 'september': 9,
    'oct': 10, 'october': 10,
    'nov': 11, 'november': 11,
    'dec': 12, 'december': 12,
}

# Nombres de meses aceptados (abreviado o completo)
month_names = (
    r'Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|'
    r'Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:t(?:ember)?)?|'
    r'Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?'
)

# Regex para D-M-Y con separador / . o -,
#    día: 1–31 con 0 opcional, mes: 1–12 o nombre inglés, año: 2 o 4 dígitos
date_pattern = re.compile(
    rf'\b(?P<d>0?[1-9]|[12]\d|3[01])'
    rf'(?P<sep>[\/\-.])'
    rf'(?P<m>(?:0?[1-9]|1[0-2])|(?:{month_names}))'
    rf'(?P=sep)'
    rf'(?P<y>\d{{2}}|\d{{4}})\b',
    flags=re.IGNORECASE
)

def normalize_year(y: str, base=2000):
    # Convierte año de 2 dígitos a 2000–2099
    return base + int(y) if len(y) == 2 else int(y)

def month_to_number(m: str):
    # Convierte mes numérico o nombre inglés a entero 1–12
    if m.isdigit():
        return int(m)
    return MONTHS_EN[m.lower()]

def extract_dates(text: str):
    dd, mm, yy = [], [], []
    for m in date_pattern.finditer(text):
        d = int(m.group('d'))
        raw_m = m.group('m')
        y = normalize_year(m.group('y'))
        try:
            mo = month_to_number(raw_m)
        except KeyError:
            # Nombre de mes no reconocido
            continue
        # Validación de rango general
        if 1 <= d <= 31 and 1 <= mo <= 12:
            dd.append(d); mm.append(mo); yy.append(y)
    return dd, mm, yy

# Buscar archivos D*.txt
files = sorted(glob.glob(os.path.join(DATA_DIR, 'D*.txt')))
if not files:
    print(f"[AVISO] No se encontraron archivos D*.txt en {DATA_DIR}")

total_d = total_m = total_y = total_n = 0
per_file_counts = []

for path in files:
    try:
        with open(path, 'r', encoding='utf-8', errors='ignore') as fh:
            txt = fh.read()
    except Exception as e:
        print(f"[ERROR] No se pudo leer {os.path.basename(path)}: {e}")
        continue

    d_list, m_list, y_list = extract_dates(txt)
    n = len(d_list)
    per_file_counts.append((os.path.basename(path), n))
    total_d += sum(d_list)
    total_m += sum(m_list)
    total_y += sum(y_list)
    total_n += n

print("Conteo por archivo:")
for name, cnt in per_file_counts:
    print(f" - {name}: {cnt}")

if total_n == 0:
    print("\n[AVISO] No se extrajeron fechas.")
else:
    avg_day = total_d / total_n
    avg_month = total_m / total_n
    avg_year = total_y / total_n
    print("\n=== Promedios globales ===")
    print(f"Día  promedio: {avg_day:.6f}")
    print(f"Mes  promedio: {avg_month:.6f}")
    print(f"Año promedio: {avg_year:.6f}")

Conteo por archivo:
 - D1.txt: 1000
 - D10.txt: 1000
 - D11.txt: 1000
 - D12.txt: 1000
 - D13.txt: 1000
 - D14.txt: 1000
 - D15.txt: 1000
 - D16.txt: 1000
 - D17.txt: 1000
 - D18.txt: 1000
 - D19.txt: 1000
 - D2.txt: 1000
 - D20.txt: 1000
 - D21.txt: 1000
 - D3.txt: 1000
 - D4.txt: 1000
 - D5.txt: 1000
 - D6.txt: 1000
 - D7.txt: 1000
 - D8.txt: 1000
 - D9.txt: 1000

=== Promedios globales ===
Día  promedio: 15.624762
Mes  promedio: 6.466476
Año promedio: 2016.686905
